In [9]:
import cv2
import albumentations as A
import os
import numpy as np

# Define augmentation pipeline
augmentation_pipeline = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.1),
    A.Rotate(limit=20, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.HueSaturationValue(p=0.5),
    A.RandomScale(scale_limit=0.2, p=0.5),
    #A.RandomCrop(width=416, height=416, p=0.5)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))


In [15]:
import shutil

def augment_data(image_dir, label_dir, output_img_dir, output_label_dir, num_augmentations=6):
    # Ensure output directories exist
    os.makedirs(output_img_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    # Loop through each image and label file
    for image_filename in os.listdir(image_dir):
        # Load image
        image_path = os.path.join(image_dir, image_filename)
        image = cv2.imread(image_path)

        # Load corresponding label file
        label_path = os.path.join(label_dir, image_filename.replace('.jpg', '.txt'))
        with open(label_path, 'r') as f:
            boxes = []
            class_labels = []
            for line in f.readlines():
                values = line.strip().split()
                class_id = int(values[0])
                bbox = list(map(float, values[1:]))
                boxes.append(bbox)
                class_labels.append(class_id)

        # Repeat augmentation `num_augmentations` times for each image
        for i in range(num_augmentations):
            augmented = augmentation_pipeline(image=image, bboxes=boxes, class_labels=class_labels)
            aug_image = augmented['image']
            aug_bboxes = augmented['bboxes']
            aug_class_labels = augmented['class_labels']

            # Save the augmented image
            aug_image_filename = f"{image_filename.split('.')[0]}_aug_{i}.jpg"
            aug_image_path = os.path.join(output_img_dir, aug_image_filename)
            cv2.imwrite(aug_image_path, aug_image)

            # Save the augmented label
            aug_label_filename = aug_image_filename.replace('.jpg', '.txt')
            aug_label_path = os.path.join(output_label_dir, aug_label_filename)
            with open(aug_label_path, 'w') as label_file:
                for label, bbox in zip(aug_class_labels, aug_bboxes):
                    label_file.write(f"{label} {' '.join(map(str, bbox))}\n")




In [17]:
# Define paths
augment_data(
    image_dir='D:/Boxing dataset/frames',
    label_dir='D:/Boxing dataset/labels',
    output_img_dir='D:/Boxing dataset/augmented_frames',
    output_label_dir='D:/Boxing dataset/augmented_labels',
    num_augmentations=6
)